# 04 Fine tuning

KLUE-NLI task 중 Pretrained model 을 사용하여 fine-tuning 하는 방법에 대해 다루겠습니다.


---

## Model load

Pretrained model을 다운 받아 fine tuning 을 진행할 수 있습니다. NLI task는 분류와 관련한 task 이므로 , `AutoModelForSequenceClassification` 클래스를 사용하겠습니다.

이때, label 개수에 대한 설정이 필요합니다.

In [13]:
dataset['train'].features['label'] # num_classes=3 

ClassLabel(num_classes=3, names=['entailment', 'neutral', 'contradiction'], names_file=None, id=None)

NLI task 는 총 3개의 label 로 구성되어 있습니다.

다음으로 모델을 불러오겠습니다. KLUE base 모델은 hugging face model hub (관련 사이트 [링크](https://huggingface.co/models)) 에 포팅되어 있으므로 model_checkpoint 경로를 정의하여 불러올 수 있습니다(`model_checkpoint = klue/bert-base` 로 사전 정의됨).


In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 3 # label 개수는 task 마다 달라질 수 있음
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

## Parameter setting
HuggingFace 에서는 `Trainer` 객체를 사용하여 학습을 진행합니다. 이때, `Trainer` 객체는 모델 학습을 위해 설정해야 하는 값이 들어있는 클래스인 `TrainingArgument`를 입력받아야 합니다.

이번 단계에서는 모델 학습을 위한 trainer 객체를 정의하는 방법에 대해 다루겠습니다.


In [15]:
import os

model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "nli") # task 별로 바꿔줄 것
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint, 모델의 checkpoint 가 저장되는 위치
    output_dir=output_dir, 
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,
    # save_steps = 500,

    # Dataset, epoch 와 batch_size 선언
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    # Optimizer
    learning_rate=2e-5, # 5e-5
    weight_decay=0.01,  # 0
    # warmup_steps=200,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,

    # Evaluation, 평가지표
    metric_for_best_model='accuracy', # task 별 평가지표 변경
    evaluation_strategy = "epoch",

    # HuggingFace Hub Upload, 모델 포팅읠 위한 인자
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging, log 기록을 살펴볼 위치, 본 노트북에서는 wandb 를 이용함
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness, 재현성을 위한 rs 설정
    seed=42,
)

`TrainingArguments` 의 여러 인자 중 필수 인자는 `output_dir` 으로 모델의 checkpoint 가 저장되는 경로를 의미합니다.
또한 task 별로 metric 지정이 필요합니다. NLI task 는 accuracy 를 평가지표로 사용합니다.

다음으로 trainer 객체를 정의하겠습니다. 우선 metric 설정이 필요합니다. datasets 라이브러리에서 제공하는 Evaluation metric의 리스트를 확인하겠습니다.

In [16]:
# metric list 확인
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

accuracy, bertscore, bleu, bleurt, cer, comet, coval, cuad, f1, gleu, glue, indic_glue, matthews_correlation, meteor, pearsonr, precision, recall, rouge, sacrebleu, sari, seqeval, spearmanr, squad, squad_v2, super_glue, wer, wiki_split, xnli


이 중, NLI 에서는 `accuracy` 를 사용합니다. 해당 평가지표를 고려하여 metric 계산을 위한 함수를 정의하겠습니다.

In [17]:
metric_name = "accuracy"
metric_accuracy = load_metric(metric_name) # metric 불러오기

# metric 계산을 위한 함수
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    ac = metric_accuracy.compute(predictions=predictions,
                                  references=labels)
    return ac

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

마지막으로 `trainer` 객체를 정의하겠습니다.

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

앞에서 tokenizer를 사용해 전처리를 했음에도 `Trainer`의 인자로 다시 넣는 이유는 패딩을 적용해서 입력 샘플들을 동일한 길이로 만들기 위해 사용하기 때문입니다. 모델에 따라 패딩에 대한 기본 설정이 다르기 때문에(왼쪽 패딩, 오른쪽 패딩, 또는 패딩 인덱스 번호 설정 등) Trainer는 이와 관련된 작업을 수행할 수 있은 tokenizer 객체를 사용합니다.

Fine-tuning 을 위한 준비가 완료되었습니다. 다음 단계에서는 fine-tuning 과 training log 를 관리하는 법에 대해 다루겠습니다.

---